# IP DATA BLOB Analysis
__authors__: ufukbombar@gmail.com

__date__: 2024-11-24

__goal__: Analyse Sandbox

__input__:
 - The measurement table as .feather file from `table_to_measurement_converter`.
 - The measurement_uuid given by the user.

 __output__:
  - The binary data for each agent under the data dir.

__notes__:
The only required fields for finding out the routing is ['probe_dst_prefix', 'reply_src_prefix'].

In [1]:
# Required packages
%pip install pandas requests python-dotenv httpx aiofiles dataclasses tqdm pyarrow

# External imports
import pandas as pd 
from dataclasses import dataclass
import json
from datetime import datetime
from tqdm import tqdm
import asyncio
import os
import ipaddress

# Internal imports
from config import config, Config
import common
import file as ff
import objects

Note: you may need to restart the kernel to use updated packages.


In [ ]:
binary_filepath = "../data/ip_data/007046a9_518e_46cb_8e70_e598b8bce831-2024-11-25 13:10:16/cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3.bin"
chunk_size = 1024 * 6 # 6 bytes equal one pair.

with open(binary_filepath, "rb+") as f:
    rows_raw = f.read(chunk_size)
    
rows_raw

b'\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xc7\x0f\xfd"\x96\xde\xc7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xc7\x0f\xfd"\x96\xde\xc7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xc7\x0f\xfd"\x96\xde\xc7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xdee\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde\xe7\x0f\xfd"\x96\xde

In [ ]:
def get_pairs(raw: bytes):
    pair_size = 6
    element_size = pair_size // 2
    num_pairs = len(raw) // pair_size
    list_return = []
    offset = 0
    for _ in range(num_pairs):
        a = raw[offset : offset + pair_size - element_size]
        b = raw[offset + element_size: offset + pair_size]